In [1]:
!pwd

/Users/thomasgiannetti/code/contatc2/green_mood_tracker/green_mood_tracker


In [6]:
from training_data import get_raw_data_notebook

In [7]:
df = get_raw_data_notebook()

/Users/thomasgiannetti/code/contatc2/green_mood_tracker/green_mood_tracker/training_data.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  sentiment140_final['polarity'] = sentiment140_final.polarity.map({4:2,0:0})
/Users/thomasgiannetti/code/contatc2/green_mood_tracker/green_mood_tracker/training_data.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  sentiment140_final['source'] = 'sentiment140'


In [8]:
df = df[0:5000]

In [9]:
df['polarity'].value_counts()

0    2245
2    1588
1    1167
Name: polarity, dtype: int64

In [12]:
from data_cleaning import clean

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/thomasgiannetti/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
df = clean(df, 'text')

In [14]:
df.shape

(5000, 4)

In [15]:
df.head()

,id,text,polarity,source
0,1467933112,angel going miss athlete weekend,0,sts_gold
1,2323395086,look though shaq getting traded cleveland play...,0,sts_gold
2,1467968979,april th isnt coming soon enough,0,sts_gold
3,1990283756,drinking mcdonalds coffee understanding someon...,0,sts_gold
4,1988884918,dissapointed taylor swift doesnt twitter,0,sts_gold


In [16]:
df['text'].isna().value_counts()

False    5000
Name: text, dtype: int64

### Bag of Words Modelling

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df['text'])

X_bow = X.toarray()
X_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [13]:
from sklearn.naive_bayes import MultinomialNB

y = df['polarity']
nb_model = MultinomialNB()
nb_model.fit(X_bow,y)
nb_model.score(X_bow,y)

0.8706

In [14]:
from sklearn.model_selection import cross_validate

modelcv= cross_validate(MultinomialNB(),X_bow, y,cv=5, scoring='accuracy')
modelcv['test_score'].mean()

0.6423871157871157

### N-Gram Modelling / 2-gram

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
tf_idf_vectorizer = TfidfVectorizer(ngram_range = (2,2))

X_1 = tf_idf_vectorizer.fit_transform(df['text'])

X_bow1 = X_1.toarray()

In [17]:
ngram_model = MultinomialNB()
ngram_model.fit(X_bow1,y)
ngram_model.score(X_bow1,y)

0.9662

In [18]:
ngram_cv = cross_validate(MultinomialNB(),X_bow1, y, cv=5, scoring='accuracy')
ngram_cv['test_score'].mean()

0.5030072016072016

### N-Gram Modelling / 3-gram

In [19]:
tf_idf_vectorizer = TfidfVectorizer(ngram_range = (3,3))

X_2 = tf_idf_vectorizer.fit_transform(df['text'])

X_bow2 = X_2.toarray()

In [20]:
ngram_model = MultinomialNB()
ngram_model.fit(X_bow2,y)
ngram_model.score(X_bow2,y)

0.9466

In [21]:
ngram_cv = cross_validate(MultinomialNB(),X_bow2, y, cv=5, scoring='accuracy')
ngram_cv['test_score'].mean()

0.4636045694045694

### Pipeline - TfidfVectorizer

In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB()),
])

parameters = {
    'tfidf__ngram_range': ((1,1), (2,2), (3,3)),
    'tfidf__max_features': (1000, 2000, 2500),
    'nb__alpha': (0.05, 0.1, 0.5, 1),}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, 
                           verbose=1, scoring = "accuracy", 
                           refit=True, cv=5)

grid_search.fit(df['text'],y)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   44.6s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...True,
        vocabulary=None)), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'tfidf__ngram_range': ((1, 1), (2, 2), (3, 3)), 'tfidf__max_features': (1000, 2000, 2500), 'nb__alpha': (0.05, 0.1, 0.5, 1)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=1)

In [23]:
grid_search.best_params_

{'nb__alpha': 0.5, 'tfidf__max_features': 2000, 'tfidf__ngram_range': (1, 1)}

In [24]:
grid_search.best_score_

0.6362

### Pipeline - CountVectorizer

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

pipeline = Pipeline([
    ('ctv', CountVectorizer()),
    ('nb', MultinomialNB()),
])

parameters = {
    'ctv__ngram_range': ((1,1), (2,2), (3,3)),
    'ctv__max_features': (1000, 2000, 2500),
    'nb__alpha': (0.05, 0.1, 0.5, 1, 2),}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, 
                           verbose=1, scoring = "accuracy", 
                           refit=True, cv=5)

grid_search.fit(df['text'],y)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   31.3s
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:   36.7s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('ctv', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'ctv__ngram_range': ((1, 1), (2, 2), (3, 3)), 'ctv__max_features': (1000, 2000, 2500), 'nb__alpha': (0.05, 0.1, 0.5, 1, 2)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=1)

In [26]:
grid_search.best_params_

{'ctv__max_features': 2000, 'ctv__ngram_range': (1, 1), 'nb__alpha': 2}

In [28]:
grid_search.best_score_

0.6428

### LDA Model

In [37]:
import nest_asyncio
import twint
import pandas as pd

nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

In [38]:
import sys
sys.path.insert(0, '../green_mood_tracker/')
from twint_class import TWINT

In [97]:
kwargs = dict(
    keywords=['nuclear'],
    cities=['London', 'Paris'],
    since = '2020-11-08 12:00:00',
    store_csv=False,
    limit=500
)

t = TWINT(**kwargs)

df_city = t.city_df()

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [87]:
df_city.shape

(711, 38)

In [44]:
df_city.head()

,cashtags,conversation_id,created_at,date,day,geo,hashtags,hour,id,language,...,trans_src,translate,tweet,urls,user_id,user_id_str,user_rt,user_rt_id,username,video
0,[],1331553301100367874,1.606302e+12,2020-11-25 11:00:20,3,,"[museumsunlocked, alaska]",11,1331553301100367874,en,...,,,Day 120 #MuseumsUnlocked &amp; Northward to #A...,[http://www.si.edu/object/NMAI_271287],1109790621299019778,1109790621299019778,,,jmosscurator,1
1,[],1331548499700310017,1.606301e+12,2020-11-25 10:41:15,3,,[],10,1331548499700310017,en,...,,,Danger One of the Biden administration’s early...,[https://www.al-monitor.com/pulse/originals/20...,3104464336,3104464336,,,AntisemitismEye,0
2,[],1331547607622168597,1.606301e+12,2020-11-25 10:37:43,3,,[iran],10,1331547607622168597,en,...,,,Is it so simple to revive the #Iran nuclear de...,[https://iranintl.com/en/world/will-biden-and-...,1028770525160660993,1028770525160660993,,,IranIntl_En,0
3,[],1331547198207782914,1.606301e+12,2020-11-25 10:36:05,3,,[],10,1331547198207782914,en,...,,,Fixed Christmas bubbles really don't work if y...,[],21016473,21016473,,,misscharlea,0
4,[],1331546060750168064,1.606300e+12,2020-11-25 10:31:34,3,,[],10,1331546060750168064,en,...,,,I have traveled to the year 2804 CE. Things ar...,[],800434781313863680,800434781313863680,,,chronomoteuse,0


In [40]:
t.keywords, t.cities, t.since

(['nuclear'], ['London'], '2020-11-10 12:00:00')

In [98]:
df_city_clean = clean(df_city, 'tweet')

In [99]:
df_city_clean['tweet'] = df_city_clean['tweet'].str.replace(kwargs['keywords'][0], '')

In [100]:
df_city_clean[['tweet']].head()

,tweet
0,don’t understand making godzilla fight king ko...
1,eu blesses romania’s bn u deal european commi...
2,four different nightclub theme park though 😂
3,apart simple fact bringing talented fresh eye ...
4,didnt realise priti patel wa head tyrannical ...


In [18]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [114]:
vectorizer = CountVectorizer(max_features=2000).fit(df_city_clean['tweet'])

data_vectorized = vectorizer.transform(df_city_clean['tweet'])

lda_model = LatentDirichletAllocation(n_components=5).fit(data_vectorized)

lda_vectors = lda_model.fit_transform(data_vectorized)

In [115]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])

In [116]:
print_topics(lda_model, vectorizer)

Topic 0:
[('iran', 41.588875228349906), ('deal', 40.37991327900828), ('would', 24.7025555328671), ('return', 19.618682291301), ('ha', 19.532753309843486), ('weapon', 17.95728922708555), ('new', 16.07718374823497), ('like', 15.784581473528887), ('war', 15.262264903830125), ('wa', 15.149388088879475)]
Topic 1:
[('power', 43.34074815127829), ('energy', 33.74606593443754), ('wa', 25.671284336034986), ('weapon', 22.20171329906541), ('reactor', 19.757141968307938), ('green', 18.955342451065658), ('uk', 18.134018254025335), ('year', 15.43029993233979), ('change', 14.061403549432615), ('need', 13.257624308687525)]
Topic 2:
[('iran', 35.7950144972336), ('weapon', 15.470488477962471), ('power', 14.695657911321344), ('plant', 14.58233663522805), ('energy', 12.636509108736556), ('deal', 11.705954988203448), ('trump', 11.170542747222735), ('work', 10.950341974833886), ('strike', 9.94878420207729), ('korea', 9.911951899198947)]
Topic 3:
[('amp', 17.443160206270683), ('family', 14.520824093701066), (

In [117]:
new_tweet = ["North Korean Leader Kim Jong Un just stated that the “Nuclear Button is on his desk at all times.” Will someone from his depleted and food starved regime please inform him that I too have a Nuclear Button, but it is a much bigger & more powerful one than his, and my Button works!"]

new_tweet_vectorized = vectorizer.transform(new_tweet)

lda_vectors = lda_model.transform(new_tweet_vectorized)

lda_vectors

array([[0.23988601, 0.01080463, 0.27947547, 0.33170802, 0.13812587]])